# HER2 One Scanner - Aperio FDA

- 5-Fold (80/20) split, No Holdout Set
- Truth = Categorical from Mean of 7 continuous scores 
- Epoch at automatic Stop when loss<.001 change 
- LeNet model, 10 layers, Dropout (0.7)

In [1]:
import numpy as np
import pandas as pd
import random
from keras.callbacks import EarlyStopping
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc, classification_report
import csv
import cv2
import scipy
import os
%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.
/usr/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#For single scanner
BASE_PATH = '/home/diam/Desktop/HER2_data_categorical/Aperio_FDA/'
#BASE PATH for working from home:
#BASE_PATH = '/home/OSEL/Desktop/HER2_data_categorical/'
#epochs = 10
batch_size = 32
num_classes = 3
#epochs = 35

## Get Data - Practice

In [3]:
#This is the version from Ravi's code:

#FDA
#X_FDA = []
#idx_FDA = []
#for index, image_filename in list(enumerate(BASE_PATH)):
#	img_file = cv2.imread(BASE_PATH + '/' + image_filename)
#	if img_file is not None:
		#img_file = smisc.imresize(arr = img_file, size = (600,760,3))
#		img_file = smisc.imresize(arr = img_file, size = (120,160,3))		
#		img_arr = np.asarray(img_file)
#		X_FDA.append(img_arr)
#		idx_FDA.append(index)

#X_FDA = np.asarray(X_FDA)
#idx_FDA = np.asarray(idx_FDA)
#random.seed(rs)
#random_id = random.sample(idx_FDA, len(idx_FDA)/2)
#random_FDA = []
#for i in random_id:
#	random_FDA.append(X_FDA[i])

#random_FDA = np.asarray(random_FDA)

## Get Data - Real

In [4]:
def get_data(folder):
    X = []
    y = []
    filenames = []

    for hclass in os.listdir(folder):
        if not hclass.startswith('.'):
            if hclass in ["1"]:
                label = 1
            else: #label must be 1 or 2
                if hclass in ["2"]:
                    label = 2
                else:
                    label = 3
            for image_filename in os.listdir(folder + hclass):
                filename = folder + hclass + '/' + image_filename
                img_file = cv2.imread(folder + hclass + '/' + image_filename)
                
                if img_file is not None:
                    img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
                    filenames.append(filename)

    X = np.asarray(X)
    y = np.asarray(y)
    z = np.asarray(filenames)
    return X,y,z

In [5]:
X, y, z = get_data(BASE_PATH)


#print(X)
#print(y)
#print(z)
print(len(X))
print(len(y))
print(len(z))

#INTEGER ENCODE
#https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
encoder = LabelEncoder()
y_cat = np_utils.to_categorical(encoder.fit_transform(y))
print(y_cat)





/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


241
241
241
[[ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.

### Old Code

In [6]:

#encoder = LabelEncoder()
#encoder.fit(y)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

#encoded_y_train = encoder.transform(y_train)
#encoded_y_test = encoder.transform(y_test)

#y_train = np_utils.to_categorical(encoded_y_train)
#y_test = np_utils.to_categorical(encoded_y_test)

In [7]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Fit Model with K-Fold X-Val

In [8]:
kf = KFold(n_splits = 241, random_state=5, shuffle=True)
print(kf.get_n_splits(y))
print(kf)


#for train_index, test_index in kf.split(y):
#    X_train, X_test = X[train_index], X[test_index]
#    print(train_index, test_index)




241
KFold(n_splits=241, random_state=5, shuffle=True)


In [9]:

oos_y = []
oos_pred = []
fold = 0


for train, test in kf.split(y_cat):
    fold+=1
    print("fold #{}".format(fold))
    
    random.shuffle(train)
    random.shuffle(test)
    
    X_train = X[train]
    y_train = y_cat[train]
    X_test = X[test]
    y_test = y_cat[test]
    
    
    model = Sequential()
    model.add(Lambda(lambda x: x * 1./255., input_shape=(120, 160, 3), output_shape=(120, 160, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(120, 160, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(3))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
    
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=25, verbose=1, mode='auto')
   
    
    model.fit(
            X_train,
            y_train,
            validation_data=(X_test,y_test),
            callbacks=[monitor],
            shuffle=True,
            batch_size=batch_size,
            verbose=0,
            epochs=1000)
        
    pred = model.predict(X_test)
        
    oos_y.append(y_test)
    #pred = np.argmax(pred)
    predx = np.argmax(pred,axis=1)
    oos_pred.append(predx)
    
        #measure the fold's accuracy
    y_compare = np.argmax(y_test,axis=1) #for accuracy calculation
    score = metrics.accuracy_score(y_compare, predx)
    print("Fold Score (accuracy): {}".format(score))
    
    print(y_test)
    #print(test)
    print(pred)
    print(predx)
    #print(z[test])
    #print(oos_y)
    
    
    #outlist = []
    
    #outline = index[0] + '\t' + str(z[test]) + '\t' + str(int(pred[0])) + '\t' + str(int(pred[1])) + '\t' + str(int(pred[2])) + '\t' + str(int(y[test])) + '\t' + str(predx) + \n'
    #outlist.append(outline)
    #with open("results.txt", "w") as f:
        #f.writelines(outlist)
        #f.writelines(z[test])

fold #1
Epoch 00042: early stopping
Fold Score (accuracy): 0.0
[[ 0.  0.  1.]]
[[ 0.08926571  0.86126703  0.04946736]]
[1]
fold #2
Epoch 00045: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  1.65378497e-21   8.51147779e-05   9.99914885e-01]]
[2]
fold #3
Epoch 00072: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[ 0.42634204  0.56138676  0.01227121]]
[1]
fold #4
Epoch 00038: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  0.00000000e+00   5.15976922e-07   9.99999523e-01]]
[2]
fold #5
Epoch 00092: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[  1.87260074e-08   8.01813722e-01   1.98186323e-01]]
[1]
fold #6
Epoch 00041: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  0.00000000e+00   1.81897704e-07   9.99999762e-01]]
[2]
fold #7
Epoch 00060: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  1.84992538e-14   2.45055277e-02   9.75494504e-01]]
[2]
fold #8
Epoch 00026: early stopping
Fold Score (accuracy): 0.0
[

Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[  4.65611811e-06   9.99991417e-01   3.96716814e-06]]
[1]
fold #63
Epoch 00040: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  2.71755464e-26   1.81089435e-02   9.81891096e-01]]
[2]
fold #64
Epoch 00095: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[  9.78732739e-09   9.97783720e-01   2.21633166e-03]]
[1]
fold #65
Epoch 00038: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  3.28207886e-14   5.40240258e-02   9.45975900e-01]]
[2]
fold #66
Epoch 00029: early stopping
Fold Score (accuracy): 0.0
[[ 0.  0.  1.]]
[[ 0.06576196  0.91363549  0.02060256]]
[1]
fold #67
Epoch 00033: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  0.00000000e+00   9.18758136e-09   1.00000000e+00]]
[2]
fold #68
Epoch 00124: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[  5.32651320e-05   9.99299645e-01   6.47141365e-04]]
[1]
fold #69
Epoch 00029: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]

Epoch 00099: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[  1.32337402e-04   9.99567091e-01   3.00552492e-04]]
[1]
fold #125
Epoch 00060: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[  2.54578458e-06   7.02200651e-01   2.97796816e-01]]
[1]
fold #126
Epoch 00049: early stopping
Fold Score (accuracy): 0.0
[[ 0.  1.  0.]]
[[  2.04331162e-23   2.64468975e-03   9.97355342e-01]]
[2]
fold #127
Epoch 00037: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  2.52138877e-27   8.35410248e-08   9.99999881e-01]]
[2]
fold #128
Epoch 00103: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[ 0.003739    0.98099661  0.01526433]]
[1]
fold #129
Epoch 00061: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[ 0.16870096  0.78496742  0.04633164]]
[1]
fold #130
Epoch 00034: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[  3.48939284e-05   6.13983035e-01   3.85982037e-01]]
[1]
fold #131
Epoch 00068: early stopping
Fold Score (accuracy):

Epoch 00174: early stopping
Fold Score (accuracy): 0.0
[[ 1.  0.  0.]]
[[  6.95633441e-02   9.30316508e-01   1.20204197e-04]]
[1]
fold #186
Epoch 00107: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[  1.41583861e-13   9.99767244e-01   2.32712118e-04]]
[1]
fold #187
Epoch 00037: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  0.00000000e+00   4.39236203e-09   1.00000000e+00]]
[2]
fold #188
Epoch 00031: early stopping
Fold Score (accuracy): 0.0
[[ 0.  0.  1.]]
[[  8.46684486e-07   5.16961694e-01   4.83037472e-01]]
[1]
fold #189
Epoch 00054: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[ 0.12317578  0.8691107   0.00771349]]
[1]
fold #190
Epoch 00053: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[ 0.25905073  0.73206562  0.0088836 ]]
[1]
fold #191
Epoch 00061: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[  6.81119934e-02   9.31622863e-01   2.65050126e-04]]
[1]
fold #192
Epoch 00052: early stopping
Fold Score (accuracy):